In [113]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [78]:
#dataframes
bossa = pd.read_csv('dataset/bossa_nova.csv')
funk = pd.read_csv('dataset/funk.csv')
gospel = pd.read_csv('dataset/gospel.csv')
sertanejo = pd.read_csv('dataset/sertanejo.csv')

In [80]:
#Atribuir colunas a generos
bossa['Gen'] = 'Bossa Nova'
funk['Gen'] = 'Funk'
gospel['Gen'] = 'Gospel'
sertanejo['Gen'] = 'Sertanejo'

In [82]:
datasets = [bossa,funk,gospel,sertanejo]

lyrics = pd.concat(datasets)

In [84]:
lyrics

,lyric,Gen
0,\r\nEu sei que vou te amar\r\nPor toda a minh...,Bossa Nova
1,\r\nOlha que coisa mais linda\r\nMais cheia d...,Bossa Nova
2,\r\nEra uma casa\r\nMuito engraçada\r\nNão ti...,Bossa Nova
3,"\r\nDe tudo, ao meu amor serei atento antes\r...",Bossa Nova
4,\r\nQuando a luz dos olhos meus\r\nE a luz do...,Bossa Nova
...,...,...
795,\r\nA luta chegou de repente\r\nE te colocou ...,Sertanejo
796,\r\nPara de mentir pra você mesmo\r\nSeu amor...,Sertanejo
797,\r\nQuadros nunca esquecem e sempre contarão\...,Sertanejo
798,\r\nVoltei era de madrugada e me assustei\r\n...,Sertanejo


In [86]:
lyrics.reset_index(drop=True,inplace=True)

In [88]:
lyrics.head()

,lyric,Gen
0,\r\nEu sei que vou te amar\r\nPor toda a minh...,Bossa Nova
1,\r\nOlha que coisa mais linda\r\nMais cheia d...,Bossa Nova
2,\r\nEra uma casa\r\nMuito engraçada\r\nNão ti...,Bossa Nova
3,"\r\nDe tudo, ao meu amor serei atento antes\r...",Bossa Nova
4,\r\nQuando a luz dos olhos meus\r\nE a luz do...,Bossa Nova


In [90]:
#Remove \n FILTRO 1
lyrics = lyrics.replace(r'\n',' ', regex=True) 
lyrics.reset_index(drop=True,inplace=True)
lyrics.head()

,lyric,Gen
0,\r Eu sei que vou te amar\r Por toda a minha ...,Bossa Nova
1,\r Olha que coisa mais linda\r Mais cheia de ...,Bossa Nova
2,\r Era uma casa\r Muito engraçada\r Não tinha...,Bossa Nova
3,"\r De tudo, ao meu amor serei atento antes\r ...",Bossa Nova
4,\r Quando a luz dos olhos meus\r E a luz dos ...,Bossa Nova


In [92]:
#Remove \r FILTRO 2
lyrics = lyrics.replace(r'\r',' ', regex=True) 
lyrics.reset_index(drop=True,inplace=True)
lyrics.head()

,lyric,Gen
0,Eu sei que vou te amar Por toda a minha vi...,Bossa Nova
1,Olha que coisa mais linda Mais cheia de gr...,Bossa Nova
2,Era uma casa Muito engraçada Não tinha te...,Bossa Nova
3,"De tudo, ao meu amor serei atento antes E ...",Bossa Nova
4,Quando a luz dos olhos meus E a luz dos ol...,Bossa Nova


In [94]:
stopwords.words('portuguese')

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

In [96]:
def processamento_texto(txt):
    # remover símbolos de pontuação, resultando em um array de caracteres
    txt = [char for char in txt if char not in string.punctuation]
    # depois, juntar os caracteres em palavras novamente e separá-los em uma lista de tokens
    txt = ''.join(txt).split()
    # por fim, remover as stopwords da lista
    txt = [word for word in txt if word.lower() not in stopwords.words('portuguese')]
    
    return txt

In [105]:
#Obter os valores Tf-idf de todas as palavras.
#Este valor indica a importância de cada palavra em relação a todo o conjunto de palavras das letras de música.
#Para isso, utiliza-se o TfidfTransformer,
#que gerará os valores.
lyric_train,lyric_test,label_train,label_test = train_test_split(lyrics['lyric'],lyrics['Gen'],test_size=0.3)

In [106]:
#Chama a função de limpeza, usa a tfidf para classicar as palavras importantes e aplica o alg svc
pipelineSVC = Pipeline([('bow',CountVectorizer(analyzer=processamento_texto)),('tfidf',TfidfTransformer()),('svc',SVC())])

In [107]:
pipelineSVC.fit(lyric_train,label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function processamento_texto at 0x0000020EAF9BEE60>)),
                ('tfidf', TfidfTransformer()), ('svc', SVC())])

In [110]:
#predição 
predictions = pipelineSVC.predict(lyric_test)

In [136]:
print(classification_report(label_test,predictions))
predictions

              precision    recall  f1-score   support

  Bossa Nova       0.85      0.85      0.85       241
        Funk       0.92      0.77      0.84       252
      Gospel       0.91      0.90      0.91       234
   Sertanejo       0.71      0.85      0.77       233

    accuracy                           0.84       960
   macro avg       0.85      0.84      0.84       960
weighted avg       0.85      0.84      0.84       960



array(['Funk', 'Sertanejo', 'Funk', 'Gospel', 'Gospel', 'Gospel',
       'Sertanejo', 'Sertanejo', 'Funk', 'Bossa Nova', 'Gospel',
       'Bossa Nova', 'Bossa Nova', 'Funk', 'Bossa Nova', 'Funk',
       'Bossa Nova', 'Gospel', 'Sertanejo', 'Sertanejo', 'Bossa Nova',
       'Sertanejo', 'Bossa Nova', 'Sertanejo', 'Sertanejo', 'Gospel',
       'Gospel', 'Gospel', 'Funk', 'Bossa Nova', 'Funk', 'Bossa Nova',
       'Sertanejo', 'Sertanejo', 'Gospel', 'Bossa Nova', 'Funk',
       'Bossa Nova', 'Funk', 'Bossa Nova', 'Sertanejo', 'Funk', 'Funk',
       'Bossa Nova', 'Bossa Nova', 'Sertanejo', 'Sertanejo', 'Bossa Nova',
       'Funk', 'Gospel', 'Bossa Nova', 'Gospel', 'Sertanejo', 'Sertanejo',
       'Sertanejo', 'Sertanejo', 'Bossa Nova', 'Gospel', 'Funk',
       'Bossa Nova', 'Funk', 'Gospel', 'Sertanejo', 'Bossa Nova', 'Funk',
       'Sertanejo', 'Bossa Nova', 'Bossa Nova', 'Gospel', 'Bossa Nova',
       'Bossa Nova', 'Bossa Nova', 'Sertanejo', 'Sertanejo', 'Funk',
       'Gospel', 'Funk', 

In [114]:
print(accuracy_score(label_test,predictions))

0.840625


In [134]:
products_list = lyrics['lyric'].values.tolist()
def partir_lista(lista):
    lista_principal = []
    for index in range(len(lista)):
        lista_final = []
        lista1 = products_list[index][:10]
        lista2 = products_list[index][10:19]
        lista3 = products_list[index][19:29]
        lista4 = products_list[index][29:]
        lista_final.append(lista1)
        lista_final.append(lista2)
        lista_final.append(lista3)
        lista_final.append(lista4)
        lista_principal.append(lista_final)
    return lista_principal

partir_lista(products_list)

[['   Eu sei ',
  'que vou t',
  'e amar  Po',
  'r toda a minha vida eu vou te amar  Em cada despedida eu vou te amar  Desesperadamente, eu sei que vou te amar  E cada verso meu será  Pra te dizer que eu sei que vou te amar  Por toda minha vida  Eu sei que vou chorar  A cada ausência tua eu vou chorar  Mas cada volta tua há de apagar  O que esta ausência tua me causou  Eu sei que vou sofrer a eterna desventura de viver  A espera de viver ao lado teu  Por toda a minha vida '],
 ['   Olha qu',
  'e coisa m',
  'ais linda ',
  ' Mais cheia de graça  É ela, menina  Que vem e que passa  Num doce balanço  A caminho do mar  Moça do corpo dourado  Do sol de Ipanema  O seu balançado é mais que um poema  É a coisa mais linda que eu já vi passar  Ah, por que estou tão sozinho?  Ah, por que tudo é tão triste?  Ah, a beleza que existe  A beleza que não é só minha  Que também passa sozinha  Ah, se ela soubesse  Que quando ela passa  O mundo inteirinho se enche de graça  E fica mais lindo  Por causa